In [ ]:
import os
import random
import requests
from pathlib import Path
from urllib.parse import quote
from duckduckgo_search import DDGS
from groq import Groq
import sys

# 初始化 Groq 客户端
GROQ_API_KEY = "your_groq_api_key"
groq_client = Groq(api_key=GROQ_API_KEY)

# 设置默认参数
ASSET_DIR = "./assets"
DATE = "2025-04-29"
CATEGORY = "攻略"
ICON = "skin"

def get_user_inputs():
    """获取用户输入的各项内容"""
    keyword = input("请输入文章主题关键词（如：一念逍遥零氪玩家玩什么职业）：")
    if not keyword:
        print("未提供关键词，退出程序")
        sys.exit(1)
    
    print("\n现在请输入AI生成文章的提示词：")
    print("默认提示词模板: 结合网上最新的相关文章或者视频，写一篇关于\"KEYWORD\"的抖音短视频文案，")
    print("时长2分钟左右，要条理清晰，重点突出，语言风趣幽默口语话，不用要旁白，直接给我文案就好，")
    print("把文章分成4个小段落，第一段写一句能够吸引人停留3秒的的开场白，就一句话就可以，")
    print("另外3段写具体内容，下面这3个小段落每个小段落加一个吸引人的小标题，每个段落标上序号，最后再加一个简单的总结\n")
    
    use_default = input("是否使用默认提示词？（y/n，默认为y）: ").lower() != 'n'
    
    if use_default:
        prompt = f"""结合网上最新的相关文章或者视频，写一篇关于"{keyword}"的抖音短视频文案，时长2分钟左右，要条理清晰，重点突出，语言风趣幽默口语话，不用要旁白，直接给我文案就好，把文章分成4个小段落，第一段写一句能够吸引人停留3秒的的开场白，就一句话就可以，另外3段写具体内容，下面这3个小段落每个小段落加一个吸引人的小标题，每个段落标上序号，最后再加一个简单的总结"""
    else:
        print("\n请输入您的自定义提示词（可以多行，输入完成后按两次回车结束）：")
        lines = []
        while True:
            line = input()
            if not line and lines and not lines[-1]:  # 连续两次回车
                break
            lines.append(line)
        
        custom_prompt = "\n".join(lines).strip()
        if not custom_prompt:
            print("未提供提示词，使用默认提示词")
            prompt = f"""结合网上最新的相关文章或者视频，写一篇关于"{keyword}"的抖音短视频文案，时长2分钟左右，要条理清晰，重点突出，语言风趣幽默口语话，不用要旁白，直接给我文案就好，把文章分成4个小段落，第一段写一句能够吸引人停留3秒的的开场白，就一句话就可以，另外3段写具体内容，下面这3个小段落每个小段落加一个吸引人的小标题，每个段落标上序号，最后再加一个简单的总结"""
        else:
            prompt = custom_prompt.replace("KEYWORD", keyword)
    
    # 获取图片相关设置
    print("\n是否需要下载相关图片？（y/n，默认为y）：")
    need_images = input().lower() != 'n'
    
    return {
        'keyword': keyword,
        'prompt': prompt,
        'need_images': need_images
    }

# 通用图片下载函数
def download_image_by_url(url: str, filename: str) -> str:
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        # 确保创建images目录
        Path("images").mkdir(exist_ok=True)
        filepath = Path("images") / filename
        with open(filepath, "wb") as f:
            f.write(response.content)
        return str(filepath)
    except Exception as e:
        print(f"下载失败：{e}")
        return ""

# 获取主图
def get_main_image(keyword: str, filename: str) -> str:
    try:
        with DDGS() as ddgs:
            results = list(ddgs.images(keyword, max_results=1))
            if results:
                image_url = results[0].get("image")
                if image_url:
                    return download_image_by_url(image_url, f"{filename}.jpg")
    except Exception as e:
        print(f"获取主图失败: {e}")
    return ""

# 下载通用插图（背景图池）
def prepare_random_images(keyword_base: str, n: int = 6):
    keywords = [
        f"{keyword_base} 游戏截图", 
        f"{keyword_base} 职业", 
        f"{keyword_base} 场景", 
        f"{keyword_base} 角色"
    ]
    image_paths = []
    try:
        with DDGS() as ddgs:
            for i in range(n):
                keyword = random.choice(keywords)
                results = list(ddgs.images(keyword, max_results=1))
                if results:
                    image_url = results[0].get("image")
                    if image_url:
                        filename = f"random_{i}.jpg"
                        path = download_image_by_url(image_url, filename)
                        if path:
                            image_paths.append(path)
    except Exception as e:
        print(f"准备随机图片失败: {e}")
    return image_paths

# 插入随机图片
def insert_random_images(text: str, random_img_paths: list) -> str:
    if not random_img_paths:
        return text
        
    paragraphs = text.split("\n\n")
    # 移除空段落
    paragraphs = [p for p in paragraphs if p.strip()]
    
    insert_count = min(len(paragraphs) - 1, len(random_img_paths), 3)  # 最多插入3张图片
    
    if insert_count <= 0:
        return text
        
    try:
        # 确保insert_positions不超出范围
        max_pos = max(1, len(paragraphs) - 1)
        positions = list(range(1, max_pos + 1))
        insert_positions = sorted(random.sample(positions, min(insert_count, len(positions))))
        
        for offset, pos in enumerate(insert_positions):
            if pos + offset < len(paragraphs):
                img_path = random_img_paths[offset % len(random_img_paths)]
                img_md = f'\n<img src="{img_path}" alt="游戏截图" style="zoom: 60%;" />\n'
                paragraphs.insert(pos + offset, img_md)
        
        return "\n\n".join(paragraphs)
    except Exception as e:
        print(f"插入图片时出错: {e}")
        return text

# 生成短视频文案
def generate_article_content(prompt: str) -> str:
    try:
        # 使用 Groq API
        print("正在通过 Groq API 生成文章内容...")
        completion = groq_client.chat.completions.create(
            model="deepseek-r1-distill-llama-70b",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_completion_tokens=1500,
            top_p=0.95,
        )
        return completion.choices[0].message.content.strip()
    except Exception as e:
        print(f"Groq API 调用失败: {e}")
        return f"生成文案失败，请检查您的网络连接和 API 密钥。错误信息: {str(e)}"

# 生成 markdown
def generate_markdown(user_inputs):
    keyword = user_inputs['keyword']
    prompt = user_inputs['prompt']
    need_images = user_inputs['need_images']
    
    # 创建资源目录
    Path(ASSET_DIR).mkdir(parents=True, exist_ok=True)
    
    # 设置文件名 - 确保文件名合法
    safe_keyword = "".join(c for c in keyword if c.isalnum() or c in " _-")
    MARKDOWN_FILE = f"{safe_keyword}.md"
    
    markdown = f"""---
title: {keyword}
icon: {ICON}
date: {DATE}
category: {CATEGORY}
star: false
# 置顶配置
# sticky: 100
---

"""
    # 获取主图
    if need_images:
        game_name = keyword.split(" ")[0] if " " in keyword else keyword
        print(f"正在下载"{game_name}"的主图...")
        main_img = get_main_image(f"{game_name} 游戏Logo", f"{game_name}_main")
        
        if main_img:
            # 使用相对路径
            markdown += f'<img src="{main_img}" alt="{game_name}" style="zoom: 67%;" />\n\n'
    
    # 生成文案内容
    print(f"正在生成关于 '{keyword}' 的文案...")
    content = generate_article_content(prompt)
    
    # 准备通用背景图并插入图片
    if need_images:
        print("正在下载相关图片...")
        game_name = keyword.split(" ")[0] if " " in keyword else keyword
        random_img_paths = prepare_random_images(game_name)
        content_with_imgs = insert_random_images(content, random_img_paths)
    else:
        content_with_imgs = content
    
    markdown += content_with_imgs
    
    # 写入文件
    with open(MARKDOWN_FILE, "w", encoding="utf-8") as f:
        f.write(markdown)
    print(f"✅ 已生成 markdown 文件：{MARKDOWN_FILE}")
    return MARKDOWN_FILE

if __name__ == "__main__":
    print("=== 欢迎使用 AI 文章生成器 ===")
    user_inputs = get_user_inputs()
    
    print(f"\n开始为关键词 '{user_inputs['keyword']}' 生成文章...")
    output_file = generate_markdown(user_inputs)
    print(f"文章已保存到 {output_file}")

开始为关键词 '--f=c:\Users\Misin\AppData\Roaming\jupyter\runtime\kernel-v35a666fade280e6515bd3fdbf20878ed194873819.json' 生成文章...
下载失败：[Errno 22] Invalid argument: 'images\\--f=c:\\Users\\Misin\\AppData\\Roaming\\jupyter\\runtime\\kernel-v35a666fade280e6515bd3fdbf20878ed194873819.json_main.jpg'
正在生成关于 '--f=c:\Users\Misin\AppData\Roaming\jupyter\runtime\kernel-v35a666fade280e6515bd3fdbf20878ed194873819.json' 的文案...
Groq API 调用失败: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}
正在下载相关图片...
✅ 已生成 markdown 文件：--fcUsersMisinAppDataRoamingjupyterruntimekernel-v35a666fade280e6515bd3fdbf20878ed194873819json.md
文章已保存到 --fcUsersMisinAppDataRoamingjupyterruntimekernel-v35a666fade280e6515bd3fdbf20878ed194873819json.md


In [ ]:
import requests
from urllib.parse import quote

# # 设置代理
# proxies = {
#     "http": "http://127.0.0.1:10809",
#     "https": "http://127.0.0.1:10809"
# }

# 定義接口 URL（將 tl=en 改為 tl=zh-TW）
url = f'https://m3u8-player.5yxy5.com/api/forward/https://translate.googleapis.com/translate_a/single?client=gtx&dt=t&sl=auto&tl=zh-TW&q={quote("中国")}'
#  proxies=proxies
# 發送 GET 請求
res = requests.get(url,)  # 別忘了添加 proxies 參數

# 提取翻譯結果
text = [te[0] for te in res.json()[0]]
print(text)


['中國']


In [ ]:
import requests
from urllib.parse import quote


# 定義接口 URL（將 tl=en 改為 tl=zh-TW）
url = f'https://m3u8-player.5yxy5.com/api/forward/https://translate.googleapis.com/translate_a/single?client=gtx&dt=t&sl=auto&tl=zh-TW&q={quote("中国")}'
# 發送 GET 請求
res = requests.get(url,)  

# 提取翻譯結果
text = [te[0] for te in res.json()[0]]
print(text)


['中國']


In [9]:
!pip install groq

Defaulting to user installation because normal site-packages is not writeable
